## (Optional) Data Cleaning and Preparation

This notebook cleans and prepares the [UCI Online Retail II Data Set](https://archive.ics.uci.edu/ml/datasets/Online+Retail+II) for use with clustering algorithms. 

Please download the latest copy of the dataset from the following UCI link and place it in a directory called `data` before executing the notebook.

File Location: https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx

*Reference*:
 * Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# create data file directory
dir_path = './data'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

Please download the latest copy of the source dataset from the UCI repo and place it in the `data` directory.

File Location: https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx

In [3]:
%pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# read the raw xlsx data
df_raw = pd.read_excel('./data/online_retail_II.xlsx',
                       engine='openpyxl')

print(df_raw.shape, df_raw.columns)

df_raw.head()

(525461, 8) Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


### Data Preparation

Clean, fill missing data (zero since it represents no sales), and restructure data to match clustering algorithm requirements.

In [5]:
# subset data needed for analysis and roll up to daily frequency
df_clean = df_raw[['StockCode', 'InvoiceDate', 'Quantity']]

df_clean['timestamp'] = df_clean['InvoiceDate'].dt.date

df_clean = (df_clean
            .groupby(['StockCode', 'timestamp'])['Quantity']
            .agg('sum')
            .reset_index())

print(df_clean.shape, df_clean.columns, df_clean.dtypes)

df_clean.head()

(272031, 3) Index(['StockCode', 'timestamp', 'Quantity'], dtype='object') StockCode    object
timestamp    object
Quantity      int64
dtype: object


/tmp/ipykernel_7678/715102072.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['timestamp'] = df_clean['InvoiceDate'].dt.date


,StockCode,timestamp,Quantity
0,10002,2009-12-01,12
1,10002,2009-12-03,7
2,10002,2009-12-04,73
3,10002,2009-12-06,49
4,10002,2009-12-07,2


In [6]:
df_pivot = df_clean.pivot(index='timestamp',
                          columns='StockCode',
                          values='Quantity')

print(df_pivot.shape, df_pivot.columns)

df_pivot.head()

(307, 4632) Index([         10002,          10080,          10109,          10120,
                10125,          10133,          10134,          10135,
                10138,          11001,
       ...
       'gift_0001_10', 'gift_0001_20', 'gift_0001_30', 'gift_0001_40',
       'gift_0001_50', 'gift_0001_60', 'gift_0001_70', 'gift_0001_80',
       'gift_0001_90',            'm'],
      dtype='object', name='StockCode', length=4632)


StockCode,10002,10080,10109,10120,10125,10133,10134,10135,10138,11001,...,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,gift_0001_60,gift_0001_70,gift_0001_80,gift_0001_90,m
timestamp,,,,,,,,,,,,,,,,,,,,,
2009-12-01,12.0,NaN,NaN,60.0,5.0,6.0,NaN,17.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12-02,NaN,1.0,NaN,10.0,NaN,NaN,NaN,20.0,96.0,69.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12-03,7.0,NaN,4.0,NaN,NaN,40.0,13.0,14.0,35.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12-04,73.0,3.0,NaN,30.0,46.0,25.0,NaN,10.0,12.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12-05,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# columns with unusual stock code data
print(list(df_pivot.columns)[-75:])

['90214D', '90214E', '90214F', '90214G', '90214H', '90214I', '90214J', '90214K', '90214L', '90214M', '90214N', '90214O', '90214P', '90214R', '90214S', '90214T', '90214U', '90214V', '90214W', '90214Y', '90214Z', 'ADJUST', 'ADJUST2', 'AMAZONFEE', 'B', 'BANK CHARGES', 'C2', 'C3', 'D', 'DCGS0003', 'DCGS0004', 'DCGS0006', 'DCGS0016', 'DCGS0027', 'DCGS0036', 'DCGS0037', 'DCGS0039', 'DCGS0041', 'DCGS0044', 'DCGS0053', 'DCGS0056', 'DCGS0058', 'DCGS0059', 'DCGS0060', 'DCGS0062', 'DCGS0066N', 'DCGS0068', 'DCGS0069', 'DCGS0070', 'DCGS0072', 'DCGS0075', 'DCGS0076', 'DCGSLBOY', 'DCGSLGIRL', 'DCGSSBOY', 'DCGSSGIRL', 'DOT', 'GIFT', 'M', 'PADS', 'POST', 'S', 'SP1002', 'TEST001', 'TEST002', 'gift_0001_10', 'gift_0001_20', 'gift_0001_30', 'gift_0001_40', 'gift_0001_50', 'gift_0001_60', 'gift_0001_70', 'gift_0001_80', 'gift_0001_90', 'm']


In [8]:
# drop columns with unusual stock code data
drop_cols = ['ADJUST', 'ADJUST2', 'AMAZONFEE', 'B', 'BANK CHARGES', 'C2', 'C3', 'D', 
             'DOT', 'GIFT', 'M', 'PADS', 'POST', 'S', 'SP1002', 'TEST001', 'TEST002',
             'gift_0001_10', 'gift_0001_20', 'gift_0001_30', 'gift_0001_40', 'gift_0001_50',
             'gift_0001_60', 'gift_0001_70', 'gift_0001_80', 'gift_0001_90', 'm']

df_pivot.drop(columns=drop_cols, inplace=True)

print(df_pivot.shape, df_pivot.columns)

df_pivot.head()

(307, 4605) Index([      10002,       10080,       10109,       10120,       10125,
             10133,       10134,       10135,       10138,       11001,
       ...
        'DCGS0068',  'DCGS0069',  'DCGS0070',  'DCGS0072',  'DCGS0075',
        'DCGS0076',  'DCGSLBOY', 'DCGSLGIRL',  'DCGSSBOY', 'DCGSSGIRL'],
      dtype='object', name='StockCode', length=4605)


StockCode,10002,10080,10109,10120,10125,10133,10134,10135,10138,11001,...,DCGS0068,DCGS0069,DCGS0070,DCGS0072,DCGS0075,DCGS0076,DCGSLBOY,DCGSLGIRL,DCGSSBOY,DCGSSGIRL
timestamp,,,,,,,,,,,,,,,,,,,,,
2009-12-01,12.0,NaN,NaN,60.0,5.0,6.0,NaN,17.0,NaN,2.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12-02,NaN,1.0,NaN,10.0,NaN,NaN,NaN,20.0,96.0,69.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12-03,7.0,NaN,4.0,NaN,NaN,40.0,13.0,14.0,35.0,2.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2009-12-04,73.0,3.0,NaN,30.0,46.0,25.0,NaN,10.0,12.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12-05,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# resample time series data and fill missing values with 0s
df_pivot.index = pd.DatetimeIndex(df_pivot.index)

df_pivot = df_pivot.resample('D').sum().fillna(0)

print(df_pivot.shape, df_pivot.columns)

df_pivot.head()

(374, 4605) Index([      10002,       10080,       10109,       10120,       10125,
             10133,       10134,       10135,       10138,       11001,
       ...
        'DCGS0068',  'DCGS0069',  'DCGS0070',  'DCGS0072',  'DCGS0075',
        'DCGS0076',  'DCGSLBOY', 'DCGSLGIRL',  'DCGSSBOY', 'DCGSSGIRL'],
      dtype='object', name='StockCode', length=4605)


StockCode,10002,10080,10109,10120,10125,10133,10134,10135,10138,11001,...,DCGS0068,DCGS0069,DCGS0070,DCGS0072,DCGS0075,DCGS0076,DCGSLBOY,DCGSLGIRL,DCGSSBOY,DCGSSGIRL
timestamp,,,,,,,,,,,,,,,,,,,,,
2009-12-01,12.0,0.0,0.0,60.0,5.0,6.0,0.0,17.0,0.0,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-12-02,0.0,1.0,0.0,10.0,0.0,0.0,0.0,20.0,96.0,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-12-03,7.0,0.0,4.0,0.0,0.0,40.0,13.0,14.0,35.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2009-12-04,73.0,3.0,0.0,30.0,46.0,25.0,0.0,10.0,12.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2009-12-05,0.0,0.0,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# transpose data to match format neeed for further processing
df_final = df_pivot.T
df_final = df_final.reset_index()

print(df_final.shape, df_final.columns)

df_final.head()

(4605, 375) Index([        'StockCode', 2009-12-01 00:00:00, 2009-12-02 00:00:00,
       2009-12-03 00:00:00, 2009-12-04 00:00:00, 2009-12-05 00:00:00,
       2009-12-06 00:00:00, 2009-12-07 00:00:00, 2009-12-08 00:00:00,
       2009-12-09 00:00:00,
       ...
       2010-11-30 00:00:00, 2010-12-01 00:00:00, 2010-12-02 00:00:00,
       2010-12-03 00:00:00, 2010-12-04 00:00:00, 2010-12-05 00:00:00,
       2010-12-06 00:00:00, 2010-12-07 00:00:00, 2010-12-08 00:00:00,
       2010-12-09 00:00:00],
      dtype='object', name='timestamp', length=375)


timestamp,StockCode,2009-12-01 00:00:00,2009-12-02 00:00:00,2009-12-03 00:00:00,2009-12-04 00:00:00,2009-12-05 00:00:00,2009-12-06 00:00:00,2009-12-07 00:00:00,2009-12-08 00:00:00,2009-12-09 00:00:00,...,2010-11-30 00:00:00,2010-12-01 00:00:00,2010-12-02 00:00:00,2010-12-03 00:00:00,2010-12-04 00:00:00,2010-12-05 00:00:00,2010-12-06 00:00:00,2010-12-07 00:00:00,2010-12-08 00:00:00,2010-12-09 00:00:00
0,10002,12.0,0.0,7.0,73.0,0.0,49.0,2.0,12.0,0.0,...,12.0,60.0,1.0,8.0,0.0,1.0,25.0,8.0,13.0,44.0
1,10080,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10109,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10120,60.0,10.0,0.0,30.0,0.0,0.0,0.0,1.0,1.0,...,10.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,12.0,0.0
4,10125,5.0,0.0,0.0,46.0,0.0,8.0,20.0,1.0,22.0,...,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,40.0,0.0


In [11]:
# back up data -> used for clustering and Forecast training in later notebooks
df_final.to_csv('./data/df_pivoted.csv.zip', index=None)

### End of processing